# Chapter 17: Pattern Matching in Depth

Pattern matching is a feature that has evolved significantly in recent versions of C#. It allows you to test the shape of data – its type, its properties, its values – in a concise and expressive way. What started as simple type checks with the `is` operator has grown into a rich language feature that includes switch expressions, property patterns, positional patterns, and even list patterns.

In this chapter, you'll learn:

- The evolution of pattern matching in C#.
- Using the **`is` expression** with type patterns, var patterns, and discard patterns.
- **Switch expressions** as a more powerful alternative to `switch` statements.
- **Relational patterns** (`<`, `>`, `<=`, `>=`) for numeric comparisons.
- **Logical patterns** (`and`, `or`, `not`) to combine patterns.
- **Property patterns** to match on property values.
- **Positional patterns** (deconstruction patterns) for tuples and custom types.
- **List patterns** (C# 11) to match sequences.
- How to combine patterns recursively.
- Practical examples and best practices.

By the end, you'll be able to write more declarative, readable code that elegantly handles complex conditional logic.

---

## 17.1 Introduction to Pattern Matching

Before C# 7, pattern matching was limited to simple type tests using `is` and `as`:

```csharp
if (obj is string)
{
    string s = (string)obj;
    Console.WriteLine(s.Length);
}
```

With C# 7, the `is` expression was extended to both test and convert in one step:

```csharp
if (obj is string s)
{
    Console.WriteLine(s.Length);
}
```

This was the beginning of pattern matching. Since then, each version of C# has added more patterns and contexts where they can be used, culminating in a comprehensive set of tools for matching data against shapes.

Pattern matching is not just about types; it's about checking whether a value conforms to a certain structure and extracting information from it.

---

## 17.2 Type Patterns and the `is` Expression

The type pattern tests whether a variable can be treated as a given type and, if so, assigns it to a new variable.

```csharp
object obj = "Hello, world!";
if (obj is string text)
{
    Console.WriteLine(text.Length); // 13
}
```

You can also use the `is` expression with a **discard** `_` if you only care about the type, not the value:

```csharp
if (obj is string)
{
    Console.WriteLine("It's a string");
}
```

But the power comes from using the variable directly.

### The `is` Operator with Not Pattern

C# 9 introduced the `not` pattern, which can be used with `is`:

```csharp
if (obj is not string)
{
    Console.WriteLine("Not a string");
}
```

This is clearer than `if (!(obj is string))`.

---

## 17.3 Switch Expressions

C# 8 introduced **switch expressions**, a more concise alternative to `switch` statements. They use the `=>` syntax and are expressions, not statements.

### Basic Switch Expression

```csharp
int number = 3;
string result = number switch
{
    1 => "One",
    2 => "Two",
    3 => "Three",
    _ => "Unknown"
};
Console.WriteLine(result); // Three
```

The `_` is the **discard pattern**, equivalent to `default` in a switch statement. It matches any value not handled by previous arms.

### Switch Expression with Type Patterns

Switch expressions shine when combined with type patterns:

```csharp
object obj = 42;
string description = obj switch
{
    int i => $"Integer: {i}",
    string s => $"String: {s}",
    double d => $"Double: {d}",
    _ => "Unknown type"
};
Console.WriteLine(description); // Integer: 42
```

### When Clauses (Guards)

You can add conditions using `when`:

```csharp
object obj = 42;
string description = obj switch
{
    int i when i > 0 => $"Positive integer: {i}",
    int i when i < 0 => $"Negative integer: {i}",
    int i => $"Zero",
    _ => "Not an integer"
};
```

---

## 17.4 Relational Patterns

C# 9 introduced relational patterns, allowing you to use `<`, `>`, `<=`, and `>=` in patterns.

```csharp
int temperature = 30;
string feeling = temperature switch
{
    < 0 => "Freezing",
    >= 0 and < 15 => "Cold",
    >= 15 and < 25 => "Mild",
    >= 25 and < 35 => "Warm",
    >= 35 => "Hot"
};
Console.WriteLine(feeling); // Warm
```

Relational patterns are often combined with logical patterns (`and`, `or`) to create ranges.

---

## 17.5 Logical Patterns: `and`, `or`, `not`

C# 9 introduced logical combinators to combine patterns.

### `and` Pattern

Both patterns must match.

```csharp
int age = 25;
string category = age switch
{
    < 13 => "Child",
    >= 13 and < 20 => "Teenager",
    >= 20 and < 65 => "Adult",
    >= 65 => "Senior"
};
```

### `or` Pattern

At least one pattern must match.

```csharp
char letter = 'a';
bool isVowel = letter switch
{
    'a' or 'e' or 'i' or 'o' or 'u' => true,
    _ => false
};
```

### `not` Pattern

Negates a pattern.

```csharp
object obj = "hello";
if (obj is not null)
{
    Console.WriteLine("Object is not null");
}
```

Logical patterns can be nested and combined arbitrarily.

---

## 17.6 Property Patterns

Property patterns allow you to match on the values of an object's properties. They are especially useful when you want to test specific aspects of an object without writing explicit `if` statements.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
    public string City { get; set; }
}

Person person = new Person { Name = "Alice", Age = 30, City = "New York" };

string description = person switch
{
    { Age: < 18 } => "Minor",
    { Age: >= 18, City: "New York" } => "Adult New Yorker",
    { Age: >= 18 } => "Adult elsewhere",
    _ => "Unknown"
};
Console.WriteLine(description); // Adult New Yorker
```

You can also nest property patterns:

```csharp
public class Address
{
    public string City { get; set; }
    public string Country { get; set; }
}

public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
    public Address Address { get; set; }
}

Person person = ...;
string location = person switch
{
    { Address: { Country: "USA", City: "New York" } } => "NYC, USA",
    { Address: { Country: "USA" } } => "Somewhere in USA",
    { Address: { Country: var country } } => $"In {country}",
    _ => "Unknown"
};
```

Property patterns make complex conditionals concise and readable.

---

## 17.7 Positional Patterns (Deconstruction Patterns)

Positional patterns rely on deconstruction. If a type has a `Deconstruct` method, you can use positional patterns to match on its components.

```csharp
public class Point
{
    public int X { get; }
    public int Y { get; }

    public Point(int x, int y) => (X, Y) = (x, y);

    public void Deconstruct(out int x, out int y) => (x, y) = (X, Y);
}

Point point = new Point(5, 3);
string quadrant = point switch
{
    (0, 0) => "Origin",
    ( > 0, > 0) => "Quadrant I",
    ( < 0, > 0) => "Quadrant II",
    ( < 0, < 0) => "Quadrant III",
    ( > 0, < 0) => "Quadrant IV",
    (0, _) => "On X axis",
    (_, 0) => "On Y axis"
};
Console.WriteLine(quadrant); // Quadrant I
```

Tuples work naturally with positional patterns:

```csharp
var tuple = (5, 3);
string result = tuple switch
{
    (0, 0) => "Origin",
    (var x, var y) when x == y => "Diagonal",
    (_, _) => "Some point"
};
```

---

## 17.8 List Patterns (C# 11)

C# 11 introduced **list patterns**, allowing you to match arrays and lists against patterns that describe their structure. You can match on elements, slices, and more.

```csharp
int[] numbers = { 1, 2, 3, 4, 5 };

string description = numbers switch
{
    [1, 2, 3, 4, 5] => "Exactly 1 to 5",
    [1, 2, .., 5] => "Starts with 1 and 2, ends with 5, anything in between",
    [1, ..] => "Starts with 1",
    [.., 5] => "Ends with 5",
    [_, _, _] => "Exactly three elements",
    [] => "Empty",
    _ => "Something else"
};
Console.WriteLine(description); // Exactly 1 to 5
```

The slice pattern `..` matches zero or more elements. You can capture the slice:

```csharp
int[] numbers = { 1, 2, 3, 4, 5 };
if (numbers is [var first, .. var rest, var last])
{
    Console.WriteLine($"First: {first}, Last: {last}, Rest length: {rest.Length}");
}
// Output: First: 1, Last: 5, Rest length: 3
```

List patterns work with any type that is *countable* and *indexable* (i.e., has a `Length` or `Count` property and an indexer).

---

## 17.9 Combining Patterns

Patterns can be nested and combined arbitrarily. For example, you can have a property pattern inside a positional pattern, or a list pattern inside a property pattern.

```csharp
record Person(string Name, int Age, List<string> Hobbies);

Person person = new Person("Alice", 30, new List<string> { "reading", "cycling" });

string info = person switch
{
    { Name: "Alice", Hobbies: [_, ..] } => "Alice has hobbies",
    { Age: > 60, Hobbies: [] } => "Senior with no hobbies",
    _ => "Other"
};
```

This flexibility allows you to express complex conditions elegantly.

---

## 17.10 Var Pattern

The `var` pattern creates a new variable and always succeeds. It's useful when you need to capture the value and use it in a `when` clause or later.

```csharp
object obj = 42;
string result = obj switch
{
    int i => $"Integer {i}",
    var x => $"Unknown type: {x?.GetType().Name ?? "null"}"
};
```

In practice, `var` patterns are less common because you can often just use a type pattern or the discard.

---

## 17.11 Putting It All Together: A Practical Example

Let's build a small expression evaluator that uses pattern matching to parse and evaluate simple arithmetic expressions.

```csharp
using System;

namespace PatternMatchingDemo
{
    // Base class for expressions
    public abstract record Expr;

    public record Constant(double Value) : Expr;
    public record Binary(string Op, Expr Left, Expr Right) : Expr;
    public record Unary(string Op, Expr Operand) : Expr;

    class Program
    {
        static void Main()
        {
            // Represent expression: (3 + 5) * (10 - 2)
            Expr expr = new Binary("*",
                new Binary("+", new Constant(3), new Constant(5)),
                new Binary("-", new Constant(10), new Constant(2)));

            double result = Evaluate(expr);
            Console.WriteLine($"Result: {result}"); // 64

            // Test simplification
            Expr simplified = Simplify(expr);
            Console.WriteLine($"Simplified: {simplified}"); // Constant(64) hopefully
        }

        static double Evaluate(Expr expr) => expr switch
        {
            Constant c => c.Value,
            Binary b => b.Op switch
            {
                "+" => Evaluate(b.Left) + Evaluate(b.Right),
                "-" => Evaluate(b.Left) - Evaluate(b.Right),
                "*" => Evaluate(b.Left) * Evaluate(b.Right),
                "/" => Evaluate(b.Left) / Evaluate(b.Right),
                _ => throw new NotSupportedException($"Operator {b.Op} not supported")
            },
            Unary u => u.Op switch
            {
                "+" => +Evaluate(u.Operand),
                "-" => -Evaluate(u.Operand),
                _ => throw new NotSupportedException($"Unary operator {u.Op} not supported")
            },
            _ => throw new ArgumentException("Unknown expression type")
        };

        static Expr Simplify(Expr expr) => expr switch
        {
            // Constant folding for binary operations
            Binary b when b.Left is Constant lc && b.Right is Constant rc 
                => new Constant(Evaluate(b)),

            // Identity: 0 + x => x, x + 0 => x
            Binary b when b.Op == "+" && b.Left is Constant { Value: 0 } 
                => Simplify(b.Right),
            Binary b when b.Op == "+" && b.Right is Constant { Value: 0 } 
                => Simplify(b.Left),

            // Identity: 1 * x => x, x * 1 => x
            Binary b when b.Op == "*" && b.Left is Constant { Value: 1 } 
                => Simplify(b.Right),
            Binary b when b.Op == "*" && b.Right is Constant { Value: 1 } 
                => Simplify(b.Left),

            // Multiply by zero => 0
            Binary b when b.Op == "*" && (b.Left is Constant { Value: 0 } || b.Right is Constant { Value: 0 })
                => new Constant(0),

            // Recursively simplify subexpressions
            Binary b => new Binary(b.Op, Simplify(b.Left), Simplify(b.Right)),
            Unary u => new Unary(u.Op, Simplify(u.Operand)),
            _ => expr
        };
    }
}
```

**Explanation:**

- We define an expression hierarchy using records (immutable by default).
- `Evaluate` uses a **switch expression** with nested switches to handle different operators. It recursively evaluates subexpressions.
- `Simplify` uses pattern matching extensively:
  - It matches on `Binary` with a `when` clause to check if both sides are constants; if so, it folds them to a single constant by evaluating.
  - It uses property patterns (`Constant { Value: 0 }`) to test for zero constants.
  - It combines multiple patterns with logical `&&` and `||` in the `when` clause.
  - It recursively simplifies subexpressions.
- This demonstrates the power of pattern matching to write clear, declarative code that would otherwise be cluttered with type casts and nested `if` statements.

---

## 17.12 Best Practices

1. **Use Pattern Matching to Replace Complex `if` Chains**  
   Pattern matching often leads to more readable code than nested `if`‑`else` or `switch` statements.

2. **Prefer Switch Expressions Over Switch Statements**  
   Switch expressions are more concise and less error‑prone (no `break` needed). They also force exhaustiveness when combined with the discard.

3. **Use Discard (`_`) for Unmatched Cases**  
   Always include a discard pattern to handle unexpected values, unless you are certain all cases are covered.

4. **Keep Patterns Simple**  
   Deeply nested patterns can be hard to read. Consider refactoring complex pattern logic into separate methods.

5. **Leverage Recursive Patterns**  
   Patterns can be nested; use this to match deep structures cleanly.

6. **Combine with `when` Clauses for Extra Conditions**  
   When a pattern alone isn't enough, add a `when` guard.

7. **Use List Patterns for Sequence Matching**  
   List patterns are a concise way to check the shape of collections.

8. **Pattern Matching is Not a Replacement for Polymorphism**  
   While pattern matching is powerful, object‑oriented polymorphism is still the right tool for many scenarios. Use pattern matching when the set of types is open or when you need to match on data, not just types.

---

## 17.13 Chapter Summary

In this chapter, you've explored the rich world of pattern matching in modern C#:

- **Type patterns** with `is` and switch expressions.
- **Switch expressions** as a compact alternative to `switch` statements.
- **Relational patterns** (`<`, `>`, etc.) for numeric comparisons.
- **Logical patterns** (`and`, `or`, `not`) to combine conditions.
- **Property patterns** to match on object properties.
- **Positional patterns** for deconstruction‑based matching.
- **List patterns** to match sequences (C# 11).
- The **var pattern** and discard.
- A practical expression evaluator and simplifier demonstrating these concepts.

Pattern matching allows you to write code that is both more expressive and safer, reducing casting and boilerplate. It's a tool you'll reach for often when dealing with complex data structures or polymorphic hierarchies.

In the next chapter, **Managing Resources – Garbage Collection & IDisposable**, we'll look under the hood at how .NET manages memory, how the garbage collector works, and how to properly release unmanaged resources using `IDisposable` and the `using` statement.

**Exercises:**

1. Write a method that takes an `object` and returns a string describing its type and value using a switch expression. Handle `int`, `string`, `double`, `bool`, and `null`.
2. Create a class hierarchy for geometric shapes (Circle, Rectangle, Triangle). Use pattern matching in a method that computes area (if shape is Circle, use radius; if Rectangle, width * height; if Triangle, use base * height / 2).
3. Use property patterns to write a method that categorizes a `Person` object based on age and city.
4. Implement a simple calculator that parses an expression string (like "3+5*2") using pattern matching? (Advanced: combine with list patterns on characters).
5. Write a function that, given a list of integers, uses list patterns to determine if the list is sorted in ascending order.

Now, prepare to dive into memory management in Chapter 18!